In [ ]:
#https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view

In [3]:
!pip install gensim

In [9]:
import gensim
gensim.__version__

'4.3.3'

In [7]:
# from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [12]:
from pathlib import Path
import zipfile, gdown
file_id = "1cuoI0G1QC-wn_5f_jqVSmRp4pT-eEmfZ"

# 저장 경로
out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)
zip_path = out_dir / "download.zip"

# 다운로드
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, str(zip_path), quiet=False)

# 압축 해제
extract_dir = out_dir
extract_dir.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1cuoI0G1QC-wn_5f_jqVSmRp4pT-eEmfZ
From (redirected): https://drive.google.com/uc?id=1cuoI0G1QC-wn_5f_jqVSmRp4pT-eEmfZ&confirm=t&uuid=fb1496f5-3ba9-4888-a291-33e53152be33
To: /content/data/download.zip
100%|██████████| 1.35G/1.35G [00:29<00:00, 46.5MB/s]


In [14]:
kv = KeyedVectors.load_word2vec_format('./data/model.bin', binary=True)

In [15]:
sim = kv.most_similar('대회')
sim

[('대회의', 0.8681818842887878),
 ('선수권', 0.8605185747146606),
 ('대회에', 0.8281852006912231),
 ('대회에서', 0.8223375678062439),
 ('월드컵', 0.8110378980636597),
 ('대회가', 0.810397744178772),
 ('대회에서는', 0.8068992495536804),
 ('대회에서도', 0.8058596849441528),
 ('대회를', 0.8058363199234009),
 ('경기대회', 0.8002544641494751)]

In [16]:
sim = kv.similarity('경기', '올림픽')
print(sim)

0.63826054


In [20]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 37.9 MB/s eta 0:00:00


In [21]:
from konlpy.tag import Okt
okt = Okt()

In [22]:
TOP_N = 200_000
cands = kv.index_to_key[:TOP_N]

def is_noun_token(word: str) -> bool:
    tags = okt.pos(word, norm=False, stem=False)
    return len(tags) > 0 and all(t[1].startswith('N') for t in tags)

noun_vocab = [w for w in cands if is_noun_token(w)]

# 명사만 담은 KeyedVectors 생성
noun_kv = KeyedVectors(vector_size=kv.vector_size)
noun_kv.add_vectors(noun_vocab, [kv[w] for w in noun_vocab])

# 저장해두고 재사용
noun_kv.save("ko_noun_only.kv")

# 사용 예시
print(noun_kv.most_similar("배터리"))

[('충전기', 0.823433518409729), ('베터리', 0.8037347197532654), ('충전', 0.7943413853645325), ('리튬', 0.791627049446106), ('마판', 0.7793197631835938), ('건전지', 0.7787209153175354), ('리튬이온', 0.7773483395576477), ('전지', 0.7672342658042908), ('전력', 0.7670044898986816), ('방진', 0.7388010025024414)]
